In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
from PIL import Image

font_path = "./font/NanumGothic.otf"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['axes.unicode_minus'] = False

# 17개 클래스별 대표 이미지 시각화
def visualize_train_classes(train_csv_path, train_img_dir, num_samples=2):
    # CSV 파일 읽기
    train_df = pd.read_csv(train_csv_path)
    
    # 클래스 이름 정의
    class_names = {
        0: "계좌번호(손글씨)", 1: "임신출산 진료비 지급 신청서", 2: "자동차 계기판", 3: "입퇴원 확인서", 4: "진단서", 
        5: "운전면허증", 6: "진료비영수증", 7: "통원/진료 확인서", 8: "주민등록증", 9: "여권", 
        10: "진료비 납입 확인서", 11: "약제비 영수증", 12: "처방전", 13: "이력서", 14: "소견서", 
        15: "자동차 등록증", 16: "자동차 번호판"
    }
    
    # 클래스별로 이미지 선택
    class_images = {}
    for class_id in range(17):
        class_df = train_df[train_df['target'] == class_id]
        selected_images = class_df['ID'].sample(num_samples).tolist()
        class_images[class_id] = selected_images
    
    # 이미지 시각화
    fig, axes = plt.subplots(4, 5, figsize=(20, 16))
    axes = axes.ravel()
    
    for idx, (class_id, image_list) in enumerate(class_images.items()):
        if idx < 17:  # 17개 클래스만 표시
            img_path = os.path.join(train_img_dir, image_list[0])
            img = Image.open(img_path)
            axes[idx].imshow(img)
            class_name = class_names[class_id]
            axes[idx].set_title(f"Class {class_id}: {class_name}", fontsize=10, fontproperties=font_prop)
            axes[idx].axis('off')
    
    plt.tight_layout()
    plt.axis('off')
    plt.show()

train_csv_path = 'data/train.csv'
train_img_dir = 'data/train/'

visualize_train_classes(train_csv_path, train_img_dir)

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import feature

def analyze_augmentations(image_folder, sample_size=100):
    images = random.sample([f for f in os.listdir(image_folder) if f.endswith('.jpg')], sample_size)
    
    brightness_vals = []
    contrast_vals = []
    edge_densities = []
    aspect_ratios = []

    for img_name in images:
        img_path = os.path.join(image_folder, img_name)
        img = np.array(Image.open(img_path).convert('RGB'))
        
        # 밝기 분석
        brightness = np.mean(img)
        brightness_vals.append(brightness)
        
        # 대비 분석
        contrast = np.std(img)
        contrast_vals.append(contrast)
        
        # 에지 밀도
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        edges = feature.canny(gray)
        edge_density = np.sum(edges) / (edges.shape[0] * edges.shape[1])
        edge_densities.append(edge_density)

        # 종횡비 (가로 세로 비율)
        aspect_ratio = img.shape[1] / img.shape[0]
        aspect_ratios.append(aspect_ratio)

    # 결과 시각화
    fig, axs = plt.subplots(2, 2, figsize=(15, 15))
    
    axs[0, 0].hist(brightness_vals, bins=20)
    axs[0, 0].set_title('Brightness Distribution')
    
    axs[0, 1].hist(contrast_vals, bins=20)
    axs[0, 1].set_title('Contrast Distribution')
    
    axs[1, 0].hist(edge_densities, bins=20)
    axs[1, 0].set_title('Edge Density Distribution')
    
    axs[1, 1].hist(aspect_ratios, bins=20)
    axs[1, 1].set_title('Aspect Ratio Distribution')
    
    plt.tight_layout()
    plt.show()

    print(f"평균 밝기: {np.mean(brightness_vals):.2f}")
    print(f"평균 대비: {np.mean(contrast_vals):.2f}")
    print(f"평균 에지 밀도: {np.mean(edge_densities):.2f}")
    print(f"평균 종횡비: {np.mean(aspect_ratios):.2f}")

analyze_augmentations('data/test/', sample_size=200)

In [ ]:
analyze_augmentations('data/test/', sample_size=3140)

네, 전체 3140개의 테스트 데이터에 대한 분석 결과를 살펴보겠습니다. 이전 200개 샘플과 비교하여 전체적인 특성을 분석해 드리겠습니다.

1. 밝기 분포 (Brightness Distribution):
   - 평균 밝기가 172.30으로, 이전 샘플(171.38)과 매우 유사합니다.
   - 분포가 오른쪽으로 치우쳐 있어, 대체로 밝은 이미지가 많습니다.
   - 200-225 범위에 가장 많은 이미지가 집중되어 있어, 전반적으로 밝은 문서 이미지임을 알 수 있습니다.

2. 대비 분포 (Contrast Distribution):
   - 평균 대비가 49.23으로, 이전 샘플(49.17)과 거의 동일합니다.
   - 분포가 20-60 사이에 집중되어 있으며, 특히 30-40 범위에 피크가 있습니다.
   - 이는 중간 정도의 대비를 가진 이미지가 많음을 나타냅니다.

3. 에지 밀도 분포 (Edge Density Distribution):
   - 평균 에지 밀도가 0.09로, 이전 샘플과 동일합니다.
   - 분포가 0.05-0.15 사이에 집중되어 있으며, 특히 0.05-0.10 범위에 피크가 있습니다.
   - 이는 대부분의 이미지가 상대적으로 낮은 에지 밀도를 가지고 있음을 의미합니다.

4. 종횡비 분포 (Aspect Ratio Distribution):
   - 평균 종횡비가 1.05로, 이전 샘플(1.01)보다 약간 높습니다.
   - 분포에서 두 개의 뚜렷한 피크가 보입니다: 0.75와 1.25 근처.
   - 이는 세로로 긴 이미지(0.75)와 가로로 긴 이미지(1.25)가 거의 비슷한 비율로 존재함을 나타냅니다.

결론 및 시사점:
1. 데이터 일관성: 200개 샘플과 전체 데이터셋의 특성이 매우 유사하여, 샘플이 전체 데이터셋을 잘 대표했음을 알 수 있습니다.
2. 이미지 밝기: 대부분의 이미지가 밝은 편이므로, 모델 학습 시 밝은 이미지에 대한 성능이 중요할 것입니다.
3. 대비: 중간 정도의 대비를 가진 이미지가 많으므로, 극단적인 대비 조정은 필요하지 않을 수 있습니다.
4. 에지 특성: 낮은 에지 밀도는 문서 이미지의 특성을 반영합니다. 텍스트 인식이나 레이아웃 분석에 중점을 둔 모델이 효과적일 수 있습니다.
5. 이미지 형태: 세로형과 가로형 문서가 비슷한 비율로 존재하므로, 모델은 두 가지 형태 모두에 대해 잘 작동해야 합니다.

데이터 증강 및 모델 학습에 대한 제안:
1. 밝기 조정: 약간의 밝기 감소 증강을 적용하여 모델의 robustness를 향상시킬 수 있습니다.
2. 대비 조정: 현재 대비 분포를 유지하면서 약간의 랜덤 대비 조정을 적용할 수 있습니다.
3. 회전 및 기울기: 종횡비 분포를 고려하여, 약간의 회전과 기울기 조정을 적용할 수 있습니다.
4. 크롭 및 패딩: 두 가지 주요 종횡비(0.75와 1.25)를 고려한 랜덤 크롭과 패딩을 적용할 수 있습니다.
5. 노이즈 추가: 낮은 에지 밀도를 고려하여, 약간의 가우시안 노이즈나 솔트 앤 페퍼 노이즈를 추가할 수 있습니다.

-----
샘플 이미지를 육안으로 봤을때 이미지 반전도 있음을 확인했고, 몇몇 이미지는 심한 정도의 증강도 있었다고 확인했음